<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Load-saved-models-for-both-people" data-toc-modified-id="Load-saved-models-for-both-people-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load saved models for both people</a></span></li><li><span><a href="#Load-Spark-Dataframe" data-toc-modified-id="Load-Spark-Dataframe-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Spark Dataframe</a></span></li><li><span><a href="#Plot-Total-Screen-Time-for-Multiple-People-by-Show" data-toc-modified-id="Plot-Total-Screen-Time-for-Multiple-People-by-Show-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Plot Total Screen Time for Multiple People by Show</a></span><ul class="toc-item"><li><span><a href="#Plot-Difference-in-Screen-Time-Between-Pairs-of-People-By-Show" data-toc-modified-id="Plot-Difference-in-Screen-Time-Between-Pairs-of-People-By-Show-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Plot Difference in Screen Time Between Pairs of People By Show</a></span></li></ul></li><li><span><a href="#Compare-Screen-Time-Over-Time-For-Multiple-People-on-a-Single-Show" data-toc-modified-id="Compare-Screen-Time-Over-Time-For-Multiple-People-on-a-Single-Show-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Compare Screen Time Over Time For Multiple People on a Single Show</a></span></li><li><span><a href="#Co-occurence-on-Screen" data-toc-modified-id="Co-occurence-on-Screen-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Co-occurence on Screen</a></span></li></ul></div>

In [ ]:
from esper.widget import *
from esper.prelude import *
from esper.identity import *
from esper.spark_identity import *
from esper.spark_util import *
from esper.validation import *

In [ ]:
people = ['Donald Trump', 'Hillary Clinton', 'Bernie Sanders']

# Load saved models for both people

In [ ]:
def load_model(name):
    print('Loading model for {}.'.format(name))
    model = FaceIdentityModel.load_from_gcs(name=name)
    imshow(tile_imgs([
        cv2.resize(x[1][0], (200, 200)) for x in model.model_params['images']], cols=10
    ))
    plt.show()
    plot_precision_and_cdf(model)
    return model

face_models = [load_model(x) for x in people]

# Load Spark Dataframe

In [ ]:
face_identities = get_face_identities()
print('Schema:', face_identities)

# Plot Total Screen Time for Multiple People by Show

In [ ]:
date_range = ['2016-01-01', '2016-11-09']
screen_time_by_canonical_show = [
    get_screen_time_by_canonical_show_spark(
        name.lower(), 
        face_identities.where(face_identities.in_commercial == False),
        date_range=date_range
    ) for name in people
]

In [ ]:
plot_screen_time_by_show(people, screen_time_by_canonical_show)

## Plot Difference in Screen Time Between Pairs of People By Show 

In [ ]:
from itertools import combinations

for i, j in combinations(range(len(people)), 2):
    plot_difference_in_screen_time_by_show(
        [x.lower() for x in [people[i], people[j]]], 
        [screen_time_by_canonical_show[i], screen_time_by_canonical_show[j]], 
        plot_proportion=False
    )

# Compare Screen Time Over Time For Multiple People on a Single Show

In [ ]:
canonical_show_name = 'MSNBC Live'
face_identities_filtered = face_identities.where(
    face_identities.canonical_show_id == CanonicalShow.objects.get(name=canonical_show_name).id
)
screen_times_by_video = [
    {
        vid : st for vid, (st, var) in get_screen_time_by_video_spark(
            name.lower(), 
            face_identities_filtered, 
            date_range=date_range
        ).items()
    } for name in people
]

In [ ]:
plot_screentime_over_time(people, canonical_show_name, 
                          screen_times_by_video)

# Co-occurence on Screen

In [ ]:
get_person_in_shot_similarity_spark(
    [x.lower() for x in people], 
    face_identities, 
    date_range=date_range
)